In [ ]:
import requests
from bs4 import BeautifulSoup
import pprint

In [ ]:
res = requests.get('https://blog.skillfactory.ru/')
print(res.status_code)

In [ ]:
pprint.pprint(res.text[:1000])

In [ ]:
url = 'https://blog.skillfactory.ru/'
params = {
    's':'python'
}
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}
req = requests.get(url, params=params, headers=headers)

pprint.pprint(req.text)

---

In [ ]:
# определяем список хабов, которые нам интересны
DESIRED_HUBS = ['python', 'big data', 'звук', 'финансы в it']

In [ ]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')

# создаем soup-объект и указываем, что работаем с html
soup = BeautifulSoup(req.text)

In [ ]:
posts = soup.find_all('article', class_='tm-articles-list__item')
pprint.pprint(posts[0])

In [ ]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    hubs = post.find_all('a', class_='tm-article-snippet__title-link')  # ищем список всех хабов в каждой статье
    for hub in hubs:
        hub_lower = hub.text.lower().replace('*', '').strip()  # при помощи атрибута text обращаемся к содержимому тэга
        #print(hub_lower)
        # ищем вхождение хотя бы одного желаемого хаба
        for element in DESIRED_HUBS:
            if element in hub_lower:
                # пост нам интересен - делаем с ним все что захотим:
                # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='tm-article-snippet__title-link')
                print(f'{title_element.text}: {title_element.get("href")}') # при помощи get достаем содержимое атрибута href
                # так как пост уже нам подошел - дальше нет смысла проверять другие его хабы
                break

---

Парсинг новостей с комерсанта по тегу угмк

In [1]:
import requests
import pprint
from bs4 import BeautifulSoup
import time
import pandas as pd

url = 'https://www.kommersant.ru/search/results'
params = {
    'datestart' : '2000-01-01',
    'dateend' : '2022-11-27',
    'sort_type' : 1,
    'page' : 1,
    'search_query': 'УГМК'
}
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

news_list = list()

for i in range(1,6):
    params['page'] = i
    req = requests.get(url, params=params, headers=headers)
    time.sleep(0.3)
    soup = BeautifulSoup(req.text)
    news = soup.find_all('article', class_='uho rubric_lenta__item')
    for element in news:
        article = element.find_all('a', class_='uho__link uho__link--overlay')
        for head in article:
            news_list.append(f'{head.text}: https://www.kommersant.ru/{head.get("href")}')
            
news_list

['Гендиректор УГМК Андрей Козицын: аукционы применимы только для нефтяных компаний: https://www.kommersant.ru//doc/353077?query=%D0%A3%D0%93%D0%9C%D0%9A',
 'Чем богата УГМК: https://www.kommersant.ru//doc/326090?query=%D0%A3%D0%93%D0%9C%D0%9A',
 'УГМК сокращает инвестпрограмму: https://www.kommersant.ru//doc/296141?query=%D0%A3%D0%93%D0%9C%D0%9A',
 '«Мы с УГМК поженились надолго»: https://www.kommersant.ru//doc/754173?query=%D0%A3%D0%93%D0%9C%D0%9A',
 '«Мы с УГМК поженились надолго»: https://www.kommersant.ru//doc/754426?query=%D0%A3%D0%93%D0%9C%D0%9A',
 'Открыт прием заявок на первую конференцию «УГМК-ТЕХНО»: https://www.kommersant.ru//doc/5283677?query=%D0%A3%D0%93%D0%9C%D0%9A',
 'Синдикат для УГМК: https://www.kommersant.ru//doc/4197234?query=%D0%A3%D0%93%D0%9C%D0%9A',
 'УГМК хочет сократить экспорт: https://www.kommersant.ru//doc/365738?query=%D0%A3%D0%93%D0%9C%D0%9A',
 'Гендиректор УГМК Андрей Козицын покинул совет директоров «Уралэлектромеди»: https://www.kommersant.ru//doc/54334